In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%pip install kaggle
%mkdir .kaggle

In [0]:
import os
os.environ['no1liekme'] = "193051f585ca0026d6a8737c25ae636f"

In [0]:
import json
token = {"username":"no1liekme","key":"193051f585ca0026d6a8737c25ae636f"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)


In [0]:
%mkdir ~/.kaggle/

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v/content

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [0]:
!mv /content/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/imdb-dataset-of-50k-movie-reviews.zip datasets/imdb.zip

In [0]:
!rm -rf datasets/lakshmi25npathi

In [0]:
%cd /content/datasets/
!pwd

In [0]:
!unzip imdb.zip

In [0]:
!rm -rf datasets/imdb.zip

In [0]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import keras


def load_data():
    data = pd.read_csv('/content/datasets/IMDB Dataset.csv')
    for i in range(len(data)):
      if data['sentiment'][i]=='negative':
         data['sentiment'][i]=0
      else:
        data['sentiment'][i]=1
    tok = Tokenizer(num_words=len(data['review']), lower=True)
    tok.fit_on_texts(data['review'])
    x = tok.texts_to_sequences(data['review'])
    x = pad_sequences(x, maxlen=200)
    y = data['sentiment']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
    return x_train, x_test, y_train, y_test


def build_model():
    model = Sequential()
    model.add(Embedding(50000, 200))
    model.add(Bidirectional(LSTM(512, return_sequences=True)))
    model.add(LSTM(512))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(loss="binary_crossentropy",
                  optimizer=keras.optimizers.Adam(),
                  metrics=["acc"])
    return model


def train(x_train, x_test, y_train, y_test):
    model_checkpoint = ModelCheckpoint('drive/My Drive/PiMA/model/model.h5',
                                       monitor='val_acc',
                                       verbose=1,
                                       save_best_only=True,
                                       mode='max')
    model = build_model()
    H = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=64, epochs=5, verbose=1,
              callbacks=[model_checkpoint])
    model.evaluate(x_test,y_test,verbose=1)


In [0]:
x_train, x_test, y_train, y_test = load_data()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
#train(x_train, x_test, y_train,y_test)

In [0]:
model = build_model()
model.load_weights('/content/drive/My Drive/PiMA/model/model.h5')
model.evaluate(x_test,y_test,verbose=1)
# prediction = model.predict(x_test)
# print(prediction)

 7136/15000 [=============>................] - ETA: 1:31